## Plotting Spotter Drift Trajectories
This notebook demonstrates plotting the latitude, longitude locations during a
 specified time period, colored by significant wave height observed.

Plotly is used to render the geographic data (continents, etc).

In [ ]:
# Import necessary packages

import pandas as pd
import plotly.graph_objects as go
import xarray as xr
import datetime

import numpy as np

import s3fs


In [ ]:
# # open the file using xarray
# filename = "spotter_data_bulk.nc"
# bulk_wave_data  = xr.load_dataset(filename)
#
# # print dataset info to see variables and the dimensions (only dimension is the observation index number)
# print(bulk_wave_data.info)

# s3_uri = "noaa-oar-hourly-gdp-pds/latest/gdp_v2.00.zarr"
s3_uri = "s3://sofar-wx-data-dev-os1/zarr/spotter_data_bulk/"

s3 = s3fs.S3FileSystem()
store = s3fs.S3Map(root=s3_uri, s3=s3, check=False)
bulk_wave_data = xr.open_zarr(store=store, consolidated=True)


In [ ]:
bulk_wave_dataframe = bulk_wave_data.to_dataframe()

# convert epoch time to datetime (easier for comparing dates)
bulk_wave_dataframe['timestamp'] = pd.to_datetime(bulk_wave_dataframe['time'], unit="s", utc=True)

# select time range of interest
# start_date = datetime.datetime(year=2022, month=5, day=25, hour=0, tzinfo=datetime.timezone.utc)
# end_date = datetime.datetime(year=2022, month=6, day=1, hour=0, tzinfo=datetime.timezone.utc)

# bulk_wave_dataframe = bulk_wave_dataframe.loc[(bulk_wave_dataframe["timestamp"] > start_date) &
#                                               (bulk_wave_dataframe["timestamp"] < end_date)]

# print(len(bulk_wave_dataframe))

Plot last data point from every Spotter reported, and color by waveheight.

In [ ]:
# Select last data available for each Spotter ID
last_data = bulk_wave_dataframe.drop_duplicates(subset="spotter_id", keep="last")


# set color thresholds
wave_min = 0  # m
wave_max = 8  # m

# make labels
marker_labels = last_data["spotter_id"] + ", Hs=" + last_data["significantWaveHeight"].round(decimals=1).astype(str) + " m"

# Use Plotly to visualize data
fig = go.Figure()
fig.add_trace(
    go.Scattergeo(
        lon=last_data["longitude"],
        lat=last_data["latitude"],
        marker=dict(
            color=last_data["significantWaveHeight"],
            colorscale="Spectral_r",
            size=10,
            cmin=wave_min,
            cmax=wave_max,
            colorbar={"title": "Observed Hs [m]"},
            line=dict(width=1, color="DarkSlateGrey"),
        ),
    text = marker_labels,
    hoverinfo = 'text'))

fig.update_geos(
    landcolor="#2B3746",
    lataxis_range=[-70, 70],
    lonaxis_range=[-180, 180],
)

fig.update_layout(title=f"Last Spotter observations")
fig.show()

In [ ]:
bulk_wave_dataframe['rounded_time'] = bulk_wave_dataframe.timestamp.round("H")

# set color thresholds
wave_min = 0  # m
wave_max = 8  # m


# Create figure
fig = go.Figure()

all_times = []
# Add traces, one for each slider step
for rounded_time, df_segmented in bulk_wave_dataframe.groupby("rounded_time"):
    all_times.append(rounded_time)

    # make labels
    marker_labels = df_segmented["spotter_id"] + ", Hs=" + df_segmented["significantWaveHeight"].round(decimals=1).astype(str) + " m"

    fig.add_trace(
        go.Scattergeo(
            visible=False,
            lon=df_segmented["longitude"],
            lat=df_segmented["latitude"],
            marker=dict(
                color=df_segmented["significantWaveHeight"],
                colorscale="Spectral_r",
                size=10,
                cmin=wave_min,
                cmax=wave_max,
                colorbar={"title": "Observed Hs [m]"},
                line=dict(width=1, color="DarkSlateGrey"),
            ),
        text = marker_labels,
        hoverinfo = 'text'))

    fig.update_geos(
        landcolor="#2B3746",
        lataxis_range=[-70, 70],
        lonaxis_range=[-180, 180],
    )



# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": f"Spotters on {all_times[i]}"}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"suffix": f"{all_times[i]}"},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()
